In [13]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [29]:
# read with pandas
#sqlitedbpath = "final_learning/database/inpatient.db"
sqlitedbpath = "../db/inpatientML.db"
conn = sqlite3.connect(sqlitedbpath)
# linelimit = 129400; # set this for testing, remove later
# df = pd.read_sql_query(f"select * from inpatient limit {linelimit};", conn)
inpatient_df = pd.read_sql_query("select * from inpatient_ML_2016_all where exclude ='N';", conn)
conn.close()
print(len(inpatient_df))
print(inpatient_df.columns)
print(inpatient_df.dtypes)

114550
Index(['drg_id', 'provider_rowid', 'provider_zip_id', 'hrr_id',
       'total_discharges', 'average_covered_charges', 'average_total_payments',
       'average_medicare_payments', 'exclude', 'weights', 'geometric_mean_los',
       'arithmetic_mean_los'],
      dtype='object')
drg_id                         int64
provider_rowid                 int64
provider_zip_id                int64
hrr_id                         int64
total_discharges               int64
average_covered_charges        int64
average_total_payments         int64
average_medicare_payments      int64
exclude                       object
weights                      float64
geometric_mean_los           float64
arithmetic_mean_los          float64
dtype: object


In [30]:
#plt.style.use('ggplot')
#scmatrix = pd.plotting.scatter_matrix(inpatient_df, figsize=(12,12),
#                             diagonal hist',
#                             marker='o',
#                             grid=True)
#plt.show()

In [31]:
# use drg_id, hrr_id,weights,geometric_mean_los, arithmetic_mean_los to predict average_covered_charges
X = inpatient_df.drop(["provider_rowid","provider_zip_id", "total_discharges", "average_total_payments", "average_covered_charges", "average_medicare_payments", "exclude"] , axis=1)
y = inpatient_df["average_covered_charges"].values.reshape(-1, 1)
print(X.shape, y.shape)
#print (X)
#print(y)


(114550, 5) (114550, 1)


In [32]:
# Split data into training and testing

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

X_train.head()


,drg_id,hrr_id,weights,geometric_mean_los,arithmetic_mean_los
92330,435,49,1.4989,4.5,6.0
56001,218,42,4.9612,11.3,14.0
59195,248,22,1.7730,4.7,6.0
33081,170,51,3.3126,5.6,7.9
62140,250,12,0.6567,2.2,2.6


In [33]:
# Create the model using LinearRegression
from sklearn.linear_model import LinearRegression
model = LinearRegression()

# Fit the model to the training data and calculate the scores for the training and testing data
model.fit(X_train, y_train)
training_score = model.score(X_train, y_train)
testing_score = model.score(X_test, y_test)

print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")



Training Score: 0.596803910599379
Testing Score: 0.5864174951936961


In [34]:
# pip install joblib
from sklearn.externals import joblib

# Save to file in the current working directory
joblib_file = "LinearRegression_model.pkl"  
#joblib.dump(model, joblib_file)

In [35]:
# Load from file

joblib_model = joblib.load(joblib_file)

# Calculate the accuracy and predictions
score = joblib_model.score(X_test, y_test)  
print("Test score: {0:.2f} %".format(100 * score))  
Ypredict = joblib_model.predict(X_test[0:1])  
print(X_test[0:1])
print(X_test[0:1].shape)
print(Ypredict, y_test[0])

Test score: 58.64 %
       drg_id  hrr_id  weights  geometric_mean_los  arithmetic_mean_los
61164     249      37    0.986                 3.2                  3.7
(1, 5)
[[32977.48804704]] [33346]


In [36]:
test_a = [ 249, 37, 0.986, 3.2, 3.7]
#print (test_a.shape)
a_df = pd.DataFrame(np.array(test_a).reshape(1, 5))
print(a_df.shape)
a_df.head()

(1, 5)


,0,1,2,3,4
0,249.0,37.0,0.986,3.2,3.7


In [37]:
test_predict = joblib_model.predict(a_df)
print(Ypredict)

[[32977.48804704]]


In [38]:
# use drg_id, hrr_id,weights,geometric_mean_los, arithmetic_mean_los, "provider_rowid"  to predict average_covered_charges
X2 = inpatient_df.drop(["provider_zip_id", "total_discharges", "average_total_payments", "average_covered_charges", "average_medicare_payments", "exclude"] , axis=1)
y2 = inpatient_df["average_covered_charges"].values.reshape(-1, 1)
print(X.shape, y.shape)
#print (X)
#print(y)

(114550, 5) (114550, 1)


In [39]:
# Split data into training and testing

from sklearn.model_selection import train_test_split

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, random_state=42)

X2_train.head()


,drg_id,provider_rowid,hrr_id,weights,geometric_mean_los,arithmetic_mean_los
92330,435,1045,49,1.4989,4.5,6.0
56001,218,1023,42,4.9612,11.3,14.0
59195,248,795,22,1.7730,4.7,6.0
33081,170,61,51,3.3126,5.6,7.9
62140,250,810,12,0.6567,2.2,2.6


In [40]:
# Create the model using LinearRegression
from sklearn.linear_model import LinearRegression
model = LinearRegression()

# Fit the model to the training data and calculate the scores for the training and testing data
model.fit(X2_train, y2_train)
training_score2 = model.score(X2_train, y2_train)
testing_score2 = model.score(X2_test, y2_test)

print(f"Training Score: {training_score2}")
print(f"Testing Score: {testing_score2}")

Training Score: 0.6077217470916345
Testing Score: 0.5964117385878531


In [41]:
# pip install joblib
from sklearn.externals import joblib

# Save to file in the current working directory
joblib_file2 = "LinearRegression_model2.pkl"  
#joblib.dump(model, joblib_file2)

In [42]:
# Load from file

joblib_model = joblib.load(joblib_file2)

# Calculate the accuracy and predictions
score = joblib_model.score(X2_test, y2_test)  
print("Test score: {0:.2f} %".format(100 * score))  
Ypredict2 = joblib_model.predict(X2_test[0:1])  
print(Ypredict2)
print(X2_test[0:1])
print(X2_test[0:1].shape)
print(Ypredict2, y2_test[0])

Test score: 59.64 %
[[38736.35778679]]
       drg_id  provider_rowid  hrr_id  weights  geometric_mean_los  \
61164     249             325      37    0.986                 3.2   

       arithmetic_mean_los  
61164                  3.7  
(1, 6)
[[38736.35778679]] [33346]


In [43]:
#test_a = [ 249, 37, 0.986, 3.2, 3.7]
#a_df = pd.DataFrame(np.array(test_a).reshape(1, 5))

#test_a = [119,325,37,1.1481,3.9,4.8]
#test_a = [119,2117,54,1.1481,3.9,4.8]
#test_a = [119,879,4,1.1481,3.9,4.8]
#test_a = [517,133,6,1.766,4.9,6.4]
test_a = [1,10,6,27.1011,30,38.6]

a_df = pd.DataFrame(np.array(test_a).reshape(1, 6))
Ypredict2 = joblib_model.predict(a_df)

print(a_df)
print(Ypredict)

     0     1    2        3     4     5
0  1.0  10.0  6.0  27.1011  30.0  38.6
[[32977.48804704]]
